# First, the review forum

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("review_forum.csv")
df.head()

,topic,page_num,post_num,text,join_date,post_date,num_likes
0,Plaquette/Guide Style Device,1,0,Hi all~ I am in the market for a new plaquett...,Sep 2017,Jun 19 2020,0
1,Plaquette/Guide Style Device,1,1,Easiest on the elbows would definitely be a G...,May 2006,Jun 19 2020,1
2,Plaquette/Guide Style Device,1,2,DMM Pivot,Mar 2019,Jun 19 2020,4
3,Plaquette/Guide Style Device,1,3,Both the Pivot and the ATC guide accept ropes...,Jun 2015,Jun 19 2020,0
4,Plaquette/Guide Style Device,1,4,Thanks Adam! As a clarification~ When climbin...,Sep 2017,Jun 19 2020,0


In [30]:
df.text[10]

' If you liked the Gigajul~ except for the relatively high friction in guide mode~ you might also want to check out the Mammut Alpine Smart. \xa0It is a bit bulky for my taste~ but guide mode is much lower friction. \xa0'

In [31]:
big_text = df.text.str.cat(sep='. ').replace("\xa0", " ")

In [32]:
big_text_list = big_text.split('. ')
len(big_text_list)

46008

In [35]:
write_df = pd.DataFrame(big_text_list, columns=["text"])
write_df.to_csv('review_forum_labels.csv', index=False)

#### I labelled 1020 rows in Google sheets, then saved the csv to my computer

In [82]:
# 0 is negative
# 1 is neutral
# 2 is positive

rev_labeled_df = pd.read_csv('review_forum_labels.csv', header = None)
rev_labeled_df.head()

,0,1
0,Hi all~ I am in the market for a new plaquette...,1
1,I currently have a Reverso 4~ but I am contemp...,1
2,I know they all operate essentially the same,1
3,When updateing my device~ I am looking for the...,2
4,I already have a GIGI that I carry on longer r...,1


# Now, the discussion forum

In [41]:
df = pd.read_csv("discussion_forum.csv")
df.head()

,topic,page_num,post_num,text,join_date,post_date,num_likes
0,Katabatic quilts and down shift,1,0,Have a hammock gear quilt right now. I reall...,Jan 2012,Jul 5 2020,1
1,Katabatic quilts and down shift,1,1,Do you roll or stuff the quilt when not in us...,Mar 2020,Jul 5 2020,0
2,Katabatic quilts and down shift,1,2,I stuff it and every night the down ends up a...,Jan 2012,Jul 6 2020,0
3,Katabatic quilts and down shift,1,3,I've had one of the Katabatic 20 degree quilt...,Jun 2016,Jul 6 2020,0
4,Adidas owning 5.10,1,0,I know this is kind of old news~ but what is ...,Jun 2017,Nov 21 2019,0


In [42]:
big_text = df.text.str.cat(sep='. ').replace("\xa0", " ")
big_text_list = big_text.split('. ')
len(big_text_list)

572786

In [45]:
# Remove duplicates but keep order
def dup_rem(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

big_text_list = dup_rem(big_text_list)

In [46]:
# # this one is too big to open up in Google Sheets
# write_df = pd.DataFrame(big_text_list, columns=["text"])
# write_df.to_csv('discussion_forum_labels.csv', index=False)

#### I labelled 3030 rows in Google sheets, then saved the csv to my computer

In [49]:
truncated_text_list = big_text_list[:5000]
write_df = pd.DataFrame(truncated_text_list, columns=["text"])
write_df.to_csv('discussion_forum_labels_small.csv', index=False)

In [54]:
disc_labeled_df = pd.read_csv('discussion_forum_labels_small.csv')
disc_labeled_df.head()

,text,1
0,Have a hammock gear quilt right now,1
1,I really like it but am annoyed with the conti...,0
2,The down always seems to end up in the end of ...,0
3,I like the pad attachment system that Katabati...,2
4,Do you roll or stuff the quilt when not in use...,1


# Let's look at what I have

In [66]:
import numpy as np

In [59]:
sum(rev_labeled_df[1][rev_labeled_df[1] != 1]) #718 non-neutral examples

718

In [64]:
sum(disc_labeled_df["1"][disc_labeled_df["1"] != 1]) #516 non-neutral examples

516

In [70]:
# where is the last row that is a 0 or 2?
max(np.where(disc_labeled_df["1"] != 1)[0]) # 3030 rows in the disc_labeled_df
max(np.where(rev_labeled_df[1] != 1)[0]) # 1017 rows in the rev_labeled_df

1017

In [78]:
disc_labeled_df = disc_labeled_df.rename(columns = {'text':"text", '1':"sentiment"})
rev_labeled_df = rev_labeled_df.rename(columns = {0:"text", 1:"sentiment"})

In [79]:
# extract only labeled rows and combine into one df
disc_test_df = disc_labeled_df[:3031]
rev_test_df = rev_labeled_df[:1018]
test_df = disc_test_df.append(rev_test_df, ignore_index=True)
test_df.head()

,text,sentiment
0,Have a hammock gear quilt right now,1
1,I really like it but am annoyed with the conti...,0
2,The down always seems to end up in the end of ...,0
3,I like the pad attachment system that Katabati...,2
4,Do you roll or stuff the quilt when not in use...,1


In [80]:
test_df.sentiment.mean()

1.0602617930353173

In [81]:
test_data = test_df.text
test_labels = test_df.sentiment

In [83]:
test_df.to_csv('labeled_forum_test.csv', index=False)